In [26]:
# How many employees does the company have?
employees = 100

# How many days are we calculating the loss?
days = 365

# On average how many emails does each employee receive weekly?
phishing_emails_per_week = 1

# How many of the phishing emails do employees click?
click_rate = 0.1

# How much money do we expect to lose with each click?
loss_per_click = 1000

In [142]:
# estimate loss due to clicking on phishing emails for a single employee
def employee_click_loss(n_days,
                        n_emails_per_week,
                        click_rate,
                        loss_per_click,
                        random_seed=None):
    import pandas as pd
    import numpy as np
    import random as rn

    # Set the random seed if provided
    if random_seed is not None:
        np.random.seed(random_seed)

    # Define the parameters for the the exponential distribution
    lambda_ = n_emails_per_week / 7  # convert rate to days

    # Generate the time intervals between phishing emails
    time_intervals = np.random.exponential(scale=1 / lambda_,
                                           size=int(lambda_ * n_days * 1.1))

    # Calculate the email arrival times as the cumulative sum of time intervals
    event_times = np.cumsum(time_intervals)

    # Select those event times that are within the time horizon
    valid_times = [
        event_time for event_time in event_times if event_time <= n_days
    ]

    # Calculate which days the emails arrived
    event_days = np.round(valid_times).astype(int)

    # Replicate probabilities
    click_rates = np.full(len(event_days), click_rate)

    # Select which emails the employee opened
    emails_opened = [
        event_day for event_day, click_rate in zip(event_days, click_rates)
        if rn.random() < click_rate
    ]

    # Calculate loss
    loss = len(emails_opened) * loss_per_click

    return {
        "time_intervals": time_intervals,
        "event_times": event_times,
        "event_days": event_days,
        "click_rates": click_rates,
        "emails_opened": emails_opened,
        "loss": loss
    }


click_loss = employee_click_loss(n_days=days,
                                 n_emails_per_week=phishing_emails_per_week,
                                 click_rate=click_rate,
                                 loss_per_click=loss_per_click)

print("Phishing emails received on days:")
print(click_loss['event_days'])

print("\nEmployee clicked on email on days:")
print(click_loss['emails_opened'])

print("\nTotal loss:")
print(click_loss['loss'])

Phishing emails received on days:
[ 23  23  26  27  40  41  43  44  46  63  89 106 111 112 118 120 125 132
 143 145 149 151 168 183 185 187 189 190 194 196 199 202 212 214 227 228
 230 239 247 263 271 273 277 282 291 301 303 309 312 317 322 327 343 347]

Employee clicked on email on days:
[168, 202, 312]

Total loss:
3000
